In [ ]:
import torch
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
%matplotlib inline

We assume all data generation/training is complete. If not done yet, please run on the terminal:
```
lasdi burgers1d.yml
```

In [ ]:
from lasdi.latent_space import Autoencoder, initial_condition_latent
from lasdi.postprocess import compute_errors

filename = 'lasdi_09_18_2024_20_51.npy'

Initialize physics solver, according to the config file `burgers1d.yml`

In [ ]:
import yaml
from lasdi.workflow import initialize_trainer
from lasdi.param import ParameterSpace

cfg_file = 'burgers1d.yml'
with open(cfg_file, 'r') as f:
    config = yaml.safe_load(f)

restart_file = np.load(filename, allow_pickle=True).item()

trainer, param_space, physics, autoencoder, sindy = initialize_trainer(config, restart_file)

In [ ]:
from lasdi.gp import fit_gps

autoencoder_param = restart_file['latent_space']['autoencoder_param']

X_train = restart_file['trainer']['X_train']

paramspace_dict = restart_file['parameters']
param_train = paramspace_dict['train_space']
param_grid = paramspace_dict['test_space']
test_meshgrid = paramspace_dict['test_meshgrid']
test_grid_sizes = paramspace_dict['test_grid_sizes']

n_init = paramspace_dict['n_init']
n_samples = 20
n_a_grid, n_w_grid = test_grid_sizes
a_grid, w_grid = test_meshgrid

physics_dict = restart_file['physics']
t_grid = physics_dict['t_grid']
x_grid = physics_dict['x_grid']
t_mesh, x_mesh = np.meshgrid(t_grid, x_grid)
Dt = physics_dict['dt']
Dx = physics_dict['dx']

# total_time = bglasdi_results['total_time']
# start_train_phase = bglasdi_results['start_train_phase']
# start_fom_phase = bglasdi_results['start_fom_phase']
# end_train_phase = bglasdi_results['end_train_phase']
# end_fom_phase = bglasdi_results['end_fom_phase']

data_test = np.load('data/data_test.npy', allow_pickle = True).item()
X_test = data_test['X_test']

time_dim, space_dim = t_grid.shape[0], x_grid.shape[0]

n_hidden = len(autoencoder_param.keys()) // 4 - 1
hidden_units = [autoencoder_param['fc' + str(i + 1) + '_e.weight'].shape[0] for i in range(n_hidden)]
n_z = autoencoder_param['fc' + str(n_hidden + 1) + '_e.weight'].shape[0]

autoencoder.load_state_dict(autoencoder_param)
coefs = restart_file['trainer']['best_coefs']
gp_dictionnary = fit_gps(param_space.train_space, coefs)

n_coef = sindy.ncoefs

from lasdi.gplasdi import sample_roms, average_rom
Zis_samples = sample_roms(autoencoder, physics, sindy, gp_dictionnary, param_grid, n_samples)
Zis_mean = average_rom(autoencoder, physics, sindy, gp_dictionnary, param_grid)

print(Zis_mean.shape)
print(Zis_samples.shape)

X_pred_mean = autoencoder.decoder(torch.Tensor(Zis_mean)).detach().numpy()
X_pred_samples = autoencoder.decoder(torch.Tensor(Zis_samples)).detach().numpy()

avg_rel_error = np.zeros(param_grid.shape[0])
for k in range(param_grid.shape[0]):
    avg_rel_error[k], _ = compute_errors(X_pred_mean[k], physics, X_test[k].numpy())

max_std = np.zeros(param_grid.shape[0])
for k in range(param_grid.shape[0]):
    max_std[k] = X_pred_samples[k].std(0).max()

avg_rel_error = avg_rel_error.reshape([n_w_grid, n_a_grid]).T
max_std = max_std.reshape([n_w_grid, n_a_grid]).T

print(avg_rel_error.shape)
print(max_std.shape)

from lasdi.gp import eval_gp
gp_pred_mean, gp_pred_std = eval_gp(gp_dictionnary, param_grid)

Plot mean and standard deviation of coefficient matrix.
For SINDy of dimension 5, the coefficient matrix has a shape of (6, 5).

In [ ]:
from lasdi.postprocess import plot_gp2d

plot_gp2d(a_grid, w_grid, gp_pred_mean.reshape([n_a_grid, n_w_grid, -1]), gp_pred_std.reshape([n_a_grid, n_w_grid, -1]),
          param_train, param_labels=['a', 'w'], plot_shape=[6, 5])

In [ ]:
from lasdi.postprocess import heatmap2d

heatmap2d(avg_rel_error * 100, a_grid[:, 0], w_grid[0], param_train, 4, param_labels=['a', 'w'], title='GPLaSDI')

In [ ]:
heatmap2d(max_std * 100, a_grid[:, 0], w_grid[0], param_train, 4, param_labels=['a', 'w'], title=r'max$_{(t,x)}\sqrt{V[\tilde{u}_{\xi^*}]}$   ($\times10^{-2}$)')

In [ ]:
from lasdi.postprocess import plot_prediction

a, w = 0.9, 1.07
param = np.array([[a, w]])
true = physics.solve(param[0])
true = true.detach().numpy().reshape([physics.grid_size[0], physics.nt]).T
scale = 1

Z = sample_roms(autoencoder, physics, sindy, gp_dictionnary, param, n_samples)

Z_mean = Z[0].mean(0)
Z_std = Z[0].std(0)

pred = autoencoder.decoder(torch.Tensor(Z)).detach().numpy()
pred_std = pred[0].std(0)

plot_prediction(param, autoencoder, physics, sindy, gp_dictionnary, n_samples, true, scale)